In [ ]:
from icesrag.utils.text_preprocess.bm25_preprocess import BM25PreProcess
from icesrag.utils.text_preprocess.strategy_pattern import TextPreprocessingEngine

In [2]:
PreProcessor = TextPreprocessingEngine(BM25PreProcess())
PreProcessor.preprocess(text="The very happy cow jumped over the delicious cheese flavored moon.")

'happy cow jumped delicious cheese flavored moon'